In [1]:
from datetime import datetime as dt  #to manipulate dates

import numpy as np  #to cleanse data
import openpyxl
import pandas as pd
import plotly.express as px  #to create interactive charts
import plotly.graph_objects as go  #to create interactive charts
from datetime import datetime

In [2]:
ytd_data = pd.read_csv(
    'D:\\Sean\\Documents\\Personal\\Budget\\YTD_downloads\\2024\\wellsfargo.csv', header=None, names=['Date','Amount', 'delete1', 'delete2', 'Description'], parse_dates=['Date']
)

del ytd_data['delete1']
del ytd_data['delete2']

ytd_data['Account'] = 'wells_fargo_checking'
ytd_data.rename(columns={'Date':'Trans. Date'}, inplace=True)
ytd_data['Date'] = pd.to_datetime(ytd_data['Trans. Date'].dt.strftime('%m/%Y'), format='%m/%Y')


In [3]:
ytd_data.tail(10)

,Trans. Date,Amount,Description,Account,Date
380,2024-01-18,-111.63,T-MOBILE PCS SVC 240117 1121101 DON LEWIS,wells_fargo_checking,2024-01-01
381,2024-01-17,-100.00,ZELLE TO LEWIS DORRY ON 01/17 REF #PP0RW6NBQX ...,wells_fargo_checking,2024-01-01
382,2024-01-17,123.48,STUBHUB CONS PAYMENTS 240117 000000017167884 9...,wells_fargo_checking,2024-01-01
383,2024-01-16,-426.16,DEPT EDUCATION STUDENT LN 240112 6PN51D9CCK1 S...,wells_fargo_checking,2024-01-01
384,2024-01-16,180.00,ZELLE FROM LEWIS JUSTINA ON 01/13 REF # PP0RVX...,wells_fargo_checking,2024-01-01
385,2024-01-11,-33.17,PURCHASE AUTHORIZED ON 01/11 ARCO#00524MEHRZ M...,wells_fargo_checking,2024-01-01
386,2024-01-11,-12.06,PURCHASE AUTHORIZED ON 01/11 ARCO#00524MEHRZ M...,wells_fargo_checking,2024-01-01
387,2024-01-11,502.15,Nationwide PAYROLL 01 HRP0009988056 Sean Lewis,wells_fargo_checking,2024-01-01
388,2024-01-02,-47.14,PURCHASE AUTHORIZED ON 01/01 A&A GAS BURLINGAM...,wells_fargo_checking,2024-01-01
389,2024-01-02,-0.50,PURCHASE AUTHORIZED ON 12/30 REDWOOD CITY PARK...,wells_fargo_checking,2024-01-01


In [15]:
# Discover Transactions
mapping = {
    # Direct Deposit
    'income': ['PAYROLL'],
    'family': ['JAYELIN', 'JUSTINA', 'SHARON', 'DORRY'],
    'purchases': ['THERAPY STORES', 'AMAZON', 'NOISEP'],
    'other_people': ['NIKA'],
    'venmo': ['VENMO', 'JONATHAN'],
    'refunds': ['PREARRANGE', 'STUBHUB CONS PAYMENTS 240117 000000017167884 9016004299-SEAN LEWIS'],
    'cell_phone': ['T-Mobile', 'TMOBILE'],
    'la_fitness': ['L A FITNESS'],
    'transfer': ['TRANSFER','MOBILE DEPOSIT : REF NUMBER :314230152812'],
    'student_loan': ['DEPT EDUCATION'],
    'dentist': ['OANH LE'],
    'care_credit': ['Synchrony Bank CC'],
    'fees': ['FEE'],
    'gas': ['SHELL', 'CHEVRON', 'ARCO', '76', 'A&A','PURCHASE AUTHORIZED ON 12/21 PALO ALTO GAS & PALO ALTO CA P000000885077547 CARD 1081'],
    'laundry': ['Cash eWithdrawal in Branch'],
    'parking': ['REDWOOD CITY PARKI REDWOOD CITY', 'STADIUM GARAGE','PURCHASE AUTHORIZED ON 10/25 STADIUM GARAGE BERKELEY CA S584300071995368 CARD 1081'],
    'ATM': ['ATM'],
    'Vincent': ['Kai Feng'],
    'GT_China': ['Candice Mclemore', 'Alipay', 'CHN', 'Macau', 'HKG'],
    'Australia': ['SYDNEY AUS', 'AUS'],
    'Mexico': ['CIUDAD', 'MEX', 'GetYourGuide'],
    'learning': ['BRITISH SWIM SCHOO'],
    'parking': [r'LAZ P.*', r'Park.*'],
    'Don': ['LEWIS JR', 'DON'],
    'general_festival': ['Stockton 9'],
    'misc': ['Lewis Sean'],
    'restaurants': ['West Town Bank', 'PIZZA HUT'],
    'movies': ['AMC'],
    'groceries': ['WEIS', 'BJS'],

}
for k, v in mapping.items():
    ytd_data.loc[ytd_data.Description.str.contains('|'.join(v), case=False), 'Short_Desc'] = k


In [16]:
ytd_data[ytd_data['Short_Desc'].isnull()].sort_values(by='Amount', ascending=False)

,index,Trans. Date,Amount,Description,Account,Date,Short_Desc,Reoccurring_Flag,Category_2


In [17]:
ytd_data.loc[ytd_data['Short_Desc'].isnull(), 'Short_Desc'] = 'unsure'

In [18]:
reocurring_list = ['cell_phone', 'la_fitness', 'student_loan','care_credit']
ytd_data.loc[ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'Yes'
ytd_data.loc[~ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'No'

mapping = {
    'Housing': [],
    'Transportation': ['parking'],
    'Food': ['groceries'],
    'Insurance': [],
    'Utilities': ['cell_phone','gas','laundry'],
    'Medical': ['medical','care_credit','dentist'],
    'Government': ['student_loan'],
    'Savings': [],
    'Personal_Spending': ['la_fitness','ATM','restaurants','learning', 'purchases'],
    'Recreation_Entertainment': ['general_festival', 'movies'],
    'Travel': ['GT_China','Australia','Mexico','air_travel', 'airport'],
    'Misc': ['unsure','misc'],
    'People': ['family','venmo','other_people','transfer','Don','Vincent'],
    'Payment_and_Interest': ['fees'],
    
    'Income':['income','refunds']
}

for k, v in mapping.items():
    ytd_data.loc[ytd_data.Short_Desc.isin(v), 'Category_2'] = k

In [19]:
ytd_data[ytd_data['Category_2'].isnull()].sort_values(by='Description', ascending=False).sort_values(by='Short_Desc',
                                                                                                     ascending=False)

,index,Trans. Date,Amount,Description,Account,Date,Short_Desc,Reoccurring_Flag,Category_2
18,18,2024-10-28,-30.0,PURCHASE AUTHORIZED ON 10/25 STADIUM GARAGE BE...,wells_fargo_checking,2024-10-01,starting_balance,No,NaN


In [20]:
x = ytd_data.groupby(['Reoccurring_Flag']).size().reset_index(name='count')

x.sort_values(by='count', ascending=False)

,Reoccurring_Flag,count
0,No,371
1,Yes,19


In [21]:
ytd_data.reset_index(inplace=True)
ytd_sorted = ytd_data.sort_values(by=['Trans. Date', 'index'])

# Define the new row with datetime objects for date columns
new_row = pd.DataFrame({
    'index': [0],  # Ensure all entries are in list format to align with DataFrame structure
    'Trans. Date': [datetime.strptime('1/1/2023', '%m/%d/%Y')],  # Matching format and conversion
    'Amount': [616.53],
    'Description': ['Balance'],
    'Account': ['wells_fargo_checking'],
    'Date': [pd.to_datetime('1/1/2023')],  # This is a string representation of the date, so it's okay
    'Short_Desc': ['Balance'],
    'Reoccurring_Flag': ['No'],
    'Category_2': ['Balance'],

}, index=[0])

# Concatenate the new_row DataFrame with the existing ytd_sorted DataFrame
ytd_sorted = pd.concat([new_row, ytd_sorted], ignore_index=True)

# Ensure 'Sort_Date' is also in datetime format for the entire DataFrame
ytd_sorted['Trans. Date'] = pd.to_datetime(ytd_sorted['Trans. Date'])
ytd_sorted

,index,Trans. Date,Amount,Description,Account,Date,Short_Desc,Reoccurring_Flag,Category_2,level_0
0,0,2023-01-01,616.53,Balance,wells_fargo_checking,2023-01-01,Balance,No,Balance,NaN
1,388,2024-01-02,-47.14,PURCHASE AUTHORIZED ON 01/01 A&A GAS BURLINGAM...,wells_fargo_checking,2024-01-01,gas,No,Utilities,388.0
2,389,2024-01-02,-0.50,PURCHASE AUTHORIZED ON 12/30 REDWOOD CITY PARK...,wells_fargo_checking,2024-01-01,parking,No,Transportation,389.0
3,385,2024-01-11,-33.17,PURCHASE AUTHORIZED ON 01/11 ARCO#00524MEHRZ M...,wells_fargo_checking,2024-01-01,gas,No,Utilities,385.0
4,386,2024-01-11,-12.06,PURCHASE AUTHORIZED ON 01/11 ARCO#00524MEHRZ M...,wells_fargo_checking,2024-01-01,gas,No,Utilities,386.0
...,...,...,...,...,...,...,...,...,...,...
386,4,2024-12-17,-442.73,DEPT EDUCATION STUDENT LN 241216 6QIV2O0DSI1 S...,wells_fargo_checking,2024-12-01,student_loan,Yes,Government,4.0
387,3,2024-12-18,-70.00,T-MOBILE PCS SVC 241217 0361737 DON LEWIS,wells_fargo_checking,2024-12-01,Don,No,People,3.0
388,1,2024-12-23,-40.80,PURCHASE AUTHORIZED ON 12/21 PALO ALTO GAS & P...,wells_fargo_checking,2024-12-01,gas,No,Utilities,1.0
389,2,2024-12-23,162.80,MOBILE DEPOSIT : REF NUMBER :314230152812,wells_fargo_checking,2024-12-01,transfer,No,People,2.0


In [22]:
ytd_sorted['CumAmount'] = ytd_sorted['Amount'].cumsum()
ytd_sorted['Year'] = ytd_sorted['Trans. Date'].dt.year

ytd_sorted

,index,Trans. Date,Amount,Description,Account,Date,Short_Desc,Reoccurring_Flag,Category_2,level_0,CumAmount,Year
0,0,2023-01-01,616.53,Balance,wells_fargo_checking,2023-01-01,Balance,No,Balance,NaN,616.53,2023
1,388,2024-01-02,-47.14,PURCHASE AUTHORIZED ON 01/01 A&A GAS BURLINGAM...,wells_fargo_checking,2024-01-01,gas,No,Utilities,388.0,569.39,2024
2,389,2024-01-02,-0.50,PURCHASE AUTHORIZED ON 12/30 REDWOOD CITY PARK...,wells_fargo_checking,2024-01-01,parking,No,Transportation,389.0,568.89,2024
3,385,2024-01-11,-33.17,PURCHASE AUTHORIZED ON 01/11 ARCO#00524MEHRZ M...,wells_fargo_checking,2024-01-01,gas,No,Utilities,385.0,535.72,2024
4,386,2024-01-11,-12.06,PURCHASE AUTHORIZED ON 01/11 ARCO#00524MEHRZ M...,wells_fargo_checking,2024-01-01,gas,No,Utilities,386.0,523.66,2024
...,...,...,...,...,...,...,...,...,...,...,...,...
386,4,2024-12-17,-442.73,DEPT EDUCATION STUDENT LN 241216 6QIV2O0DSI1 S...,wells_fargo_checking,2024-12-01,student_loan,Yes,Government,4.0,870.42,2024
387,3,2024-12-18,-70.00,T-MOBILE PCS SVC 241217 0361737 DON LEWIS,wells_fargo_checking,2024-12-01,Don,No,People,3.0,800.42,2024
388,1,2024-12-23,-40.80,PURCHASE AUTHORIZED ON 12/21 PALO ALTO GAS & P...,wells_fargo_checking,2024-12-01,gas,No,Utilities,1.0,759.62,2024
389,2,2024-12-23,162.80,MOBILE DEPOSIT : REF NUMBER :314230152812,wells_fargo_checking,2024-12-01,transfer,No,People,2.0,922.42,2024


In [23]:
ytd_sorted['Counter'] = ytd_sorted.groupby('Short_Desc').cumcount() + 1
ytd_sorted

,index,Trans. Date,Amount,Description,Account,Date,Short_Desc,Reoccurring_Flag,Category_2,level_0,CumAmount,Year,Counter
0,0,2023-01-01,616.53,Balance,wells_fargo_checking,2023-01-01,Balance,No,Balance,NaN,616.53,2023,1
1,388,2024-01-02,-47.14,PURCHASE AUTHORIZED ON 01/01 A&A GAS BURLINGAM...,wells_fargo_checking,2024-01-01,gas,No,Utilities,388.0,569.39,2024,1
2,389,2024-01-02,-0.50,PURCHASE AUTHORIZED ON 12/30 REDWOOD CITY PARK...,wells_fargo_checking,2024-01-01,parking,No,Transportation,389.0,568.89,2024,1
3,385,2024-01-11,-33.17,PURCHASE AUTHORIZED ON 01/11 ARCO#00524MEHRZ M...,wells_fargo_checking,2024-01-01,gas,No,Utilities,385.0,535.72,2024,2
4,386,2024-01-11,-12.06,PURCHASE AUTHORIZED ON 01/11 ARCO#00524MEHRZ M...,wells_fargo_checking,2024-01-01,gas,No,Utilities,386.0,523.66,2024,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,4,2024-12-17,-442.73,DEPT EDUCATION STUDENT LN 241216 6QIV2O0DSI1 S...,wells_fargo_checking,2024-12-01,student_loan,Yes,Government,4.0,870.42,2024,6
387,3,2024-12-18,-70.00,T-MOBILE PCS SVC 241217 0361737 DON LEWIS,wells_fargo_checking,2024-12-01,Don,No,People,3.0,800.42,2024,17
388,1,2024-12-23,-40.80,PURCHASE AUTHORIZED ON 12/21 PALO ALTO GAS & P...,wells_fargo_checking,2024-12-01,gas,No,Utilities,1.0,759.62,2024,21
389,2,2024-12-23,162.80,MOBILE DEPOSIT : REF NUMBER :314230152812,wells_fargo_checking,2024-12-01,transfer,No,People,2.0,922.42,2024,4


In [24]:
columns = ['Trans. Date', 'Description', 'Amount', 'Category', 'Short_Desc', 'Reoccurring_Flag',
                     'Category_2', 'Date', 'Year', 'Sort_Date', 'CumAmount', 'Counter', 'Account']
ytd_sorted['Category'] = 'NA'
ytd_sorted['Sort_Date'] = ytd_sorted['Trans. Date']

wf = ytd_sorted[columns]

In [25]:
wf.to_excel('D:\\Sean\\Documents\\Personal\\Budget\\2024_output\\WF_Curated_Bills.xlsx')